In [1]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [3]:
base = automap_base()
base.prepare(engine)

In [4]:
base.classes

In [5]:
base.classes.keys()

['measurement', 'station']

In [6]:
measurement = base.classes.measurement
station = base.classes.station
session = Session(engine)

## Precipitation Analysis

In [7]:
# Find the most recent date in the dataset.
recent_date = session.query(measurement.date).order_by(measurement.date.desc()).first()
recent_date

('2017-08-23',)

In [8]:
recent_datetwo = session.query(func.max(measurement.date)).first()
recent_datetwo

('2017-08-23',)

In [9]:
# Using that date, get the previous 12 months of precipitation data by querying the previous 12 months of data.
last_year = dt.date(2017,8,23)-dt.timedelta(days=365)
last_year

datetime.date(2016, 8, 23)

In [ ]:
# Select only the "date" and "prcp" values.
question_two = session.query(measurement.date, measurement.prcp).filter(measurement.date > last_year).all()
question_two

In [ ]:
# Load the query results into a Pandas DataFrame. Explicitly set the column names.
date_pcp = pd.DataFrame(question_two)
date_pcp
#line plot later

In [ ]:
# Sort the DataFrame values by "date".
date_sort = date_pcp.sort_values("date")
date_sort

In [ ]:
# Plot the results by using the DataFrame plot method
# plt.plot(date_sort["date"], date_sort["prcp"]) #matplotlib method 1
# x_axis = date_sort["date"]
# y_axis = date_sort["prcp"]
# plt.plot(x_axis, y_axis) #matplotlib method 2
date_sort.plot("date", "prcp")
plt.xticks(rotation=90)
plt.ylabel("Precipitation")
plt.title("Hawaii Date and Precipitation")
plt.show()

In [ ]:
# Use Pandas to print the summary statistics for the precipitation data.
date_sort.describe()

## Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset.
total_stations = session.query(station.station).count()
print(f"There are {total_stations} total stations in Hawaii")

In [ ]:
# Design a query to find the most-active stations (that is, the stations that have the most rows). 
# To do so, complete the following steps:
# List the stations and observation counts in descending order.
# HINT: You’ll need to use the func.count function in your query.
# Answer the following question: which station id has the greatest number of observations?
active_station = session.query(measurement.station, func.count(measurement.station)).group_by(measurement.station).\
    order_by(func.count(measurement.station).desc()).all()
print(active_station)
print(f"The station ID of {active_station[0][0]} has the greatest number of observations.")

In [ ]:
# Design a query that calculates the lowest, highest, and average temperatures that filters on 
# the most-active station id found in the previous query.
most_active_stat = session.query(measurement.station, func.min(measurement.tobs),
                                func.max(measurement.tobs), func.avg(measurement.tobs)).\
    filter(measurement.station == active_station[0][0]).first()
most_active_stat

In [ ]:
# Design a query to get the previous 12 months of temperature observation (TOBS) data. 
# Filter by the station that has the greatest number of observations.
# Query the previous 12 months of TOBS data for that station.
most_active_data = session.query(measurement.tobs).\
    filter(measurement.station == active_station[0][0]).\
    filter(measurement.date > last_year).all()
most_active_data

In [ ]:
most_active = pd.DataFrame(most_active_data)
most_active

In [ ]:
# Plot the results as a histogram with bins=12
most_active.plot.hist(bins = 12)
plt.xlabel("Temperature")
plt.title("Temperatures for Station USC00519281")
plt.show()

In [ ]:
# Close your session.
session.close()